In [ ]:
import os
from download import download

url = "http://vam.wfp.org/sites/data/WFPVAM_FoodPrices_05-12-2017.csv"
file_name = 'WFPVAM_FoodPrices_05-12-2017.csv'

def get_dataset(url, file_name):
    if not os.path.fexists(file_name):
        download(url, file_name)
    

    
get_dataset(url, file_name)
    

  0%|          | 17.0/89.9k [00:00<10:36, 141KB/s]

 45%|████▍     | 40.3k/89.9k [00:17<00:21, 2.29kKB/s]